# Chapter 5. 이터레이터와 제너레이터

In [1]:
def fibonacci_list(num_items):
    numbers = []
    a, b = 0, 1
    while len(numbers) < num_items:
        numbers.append(a)
        a, b = b, a + b
    return numbers                   # return을 만나면 함수는 종료된다.


def fibonacci_gen(num_items):
    a, b = 0, 1
    while num_items:
        yield a                      # yield는 a값을 반환하고, 대기상태로 바뀐 다음 함수 밖의 코드를 실행하다가
        a, b = b, a + b              # 다음 값 요청이 들어오면 이전 상태를 유지한 채로 실행을 재개하여 새로운 값을 반환한다.
        num_items -= 1

In [2]:
def test_fibonacci_list():
    for i in fibonacci_list(100_000):
        pass
    

def test_fibonacci_gen():
    for i in fibonacci_gen(100_000):
        pass

In [4]:
%timeit test_fibonacci_list()

472 ms ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%load_ext memory_profiler
%memit test_fibonacci_list()

peak memory: 472.06 MiB, increment: 391.25 MiB


In [7]:
%timeit test_fibonacci_gen()

138 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%load_ext memory_profiler
%memit test_fibonacci_gen()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 81.78 MiB, increment: 0.00 MiB


**예제 5-1** 파이썬 for 루프 재구성하기

In [ ]:
# 다음 파이썬 루프는
for i in object:                 # object에 fibonacci_list를 사용하면 리스트를 먼저 만든 뒤 이터레이터를 만든다.
    do_work(i)                   # fibonacci_gen은 이미 그 자체로 이터레이터이다. 

# 아래 코드와 같다.
object_iterator = iter(object)
while True:
    try:
        i = next(object_iterator)
    except StopIteration:
        break
    else:
        do_work(i)

In [9]:
%memit len([n for n in fibonacci_gen(100_000) if n % 3 == 0])
# 피보나치 수열을 만들려고 여전히 fibonacci_gen을 제너레이터로 사용하지만,
# 3으로 나뉘는 값을 배열에 저장하고 해당 배열에서 길이만 알아낸 다음에 데이터를 버린다.
# 리스트 내포는 [<값> for <항목> in <배열> if <조건>] 문법을 사용하고, 모든 <값>이 있는 리스트를 생성한다.

peak memory: 193.13 MiB, increment: 111.48 MiB


In [11]:
%memit sum(1 for n in fibonacci_gen(100_000) if n % 3 == 0)
# 제너레이터에는 length 속성이 없으므로 위와 같이 코드를 최적화할 수 있다.
# 제너레이터 내포는 (<값> for <항목> in <배열> if <조건>) 문법을 사용하고, <값>을 생성하는 제너레이터를 만든다.

peak memory: 81.80 MiB, increment: 0.31 MiB


## 5.1 이터레이터로 무한급수 표현하기

In [12]:
def fibonacci():
    i, j = 0, 1
    while True:
        yield j
        i, j = j, i + j

In [ ]:
def fibonacci_navie():
    i, j = 0, 1
    count = 0
    while j <= 5000:
        if j % 2:
            count += 1
        i, j = j, i + j
    return count


def fibonacci_transform():
    count = 0
    for f in fibonacci():
        if f > 5000:
            break
        if f % 2:
            count += 1
    return count

from itertools import takewhile
def fibonacci_succinct():
    first_5000 = takewhile(lambda x: x <= 5000, fibonacci())
    return sum(1 for x in first_5000 if x % 2)

위의 방법은 모두 속도와 메모리 사용량이 비슷하다. 하지만 fibonacci_transform 함수가 몇 가지 면에서 유리하다. <br>
<br>
첫째, fibonacci_succinct보다 더 자세히 풀어썼으므로 다른 개발자가 디버깅하거나 이해하기 쉽다. itertools를 사용할 때는 조심해야 한다. <br>
<br>
fibonacci_ naive는 한 번에 여러 작업을 하므로 실제 어떤 계산을 수행하는지 알아보기 힘들다. 반면, 제너레이터 함수는 다른 계산의 영향을 받지 않으면서 피보나치 수열을 순회한다는 사실을 쉽게 알 수 있다. <br>
<br>
fibonacci_transform 함수가 더 일반화하기 쉽다. <br>
<br>
fibonacci_transform과 fibonacci_succinct의 또 다른 장점은 계산 과정을 데이터의 생성과 변형, 두 단계로 수분해서 생각하도록 해준다는 점이다. fibonacci 함수가 데이터를 생성하면 fibonacci_transfor 함수가 이를 변경한다.

## 5.2 제너레이터의 지연 계산

현재의 값만 필요한 경우에 제너레이터가 메모리 사용 측면에서 유리하다. 제너레이터를 사용한 피보나치 수열 계산에서도 현잿값만 사용할 뿐, 수열의 다른 값을 참조할 수는 없다. 이런 특징 때문에 가끔 제너레이터를 사용하기 어려운데, 이럴 때 표준 라이브러리인 itertools가 도움이 된다. 대표적으로 다음과 같은 함수들을 제공한다.

- islice: 제너레이터에 대한 슬라이싱 기능을 제공한다.
- chain: 여러 제너레이터를 연결할 수 있다.
- takewhile: 제너레이터 종료 조건을 추가할 수 있다.
- cycle: 유한한 제너레이터를 계속 반복하여 무한 제너레이터로 동작하도록 한다.

**예제 5-2** 필요한 때만 데이터 읽기

In [ ]:
from random import normalvariate, randint
from itertools import count
from datetime import datetime

def read_data(filename):
    with open(filename) as fd:
        for line in fd:
            data = line.strip().split(",")
            timestamp, value = map(int, data)
            yield datetime.fromtimestamp(timestamp), value


def read_fake_data(filename):
    for timestamp in count():
        #  We insert an anomalous data point approximately once a week
        if randint(0, 7 * 60 * 60 * 24 - 1) == 1:
            value = 100
        else:
            value = normalvariate(0, 1)
        yield datetime.fromtimestamp(timestamp), value

**예제 5-3** 데이터 그룹 만들기

In [ ]:
for itertools import groupby

def groupby_day(iterable):
    key = lambda row: row[0].day
    for day, data_group in groupby(iterable, key):
        yield list(data_group)

**예제 5-4** 제너레이터 기반의 특이점 찾기

In [ ]:
from scipy.stats import normaltest
for itertools import filterfalse

def is_normal(data, threshold=1e-3):
    _, values = zip(*data)
    k2, p_value = normaltest(values)
    if p_value < threshold:
        return False
    return True


def filter_anomalous_groups(data):
    yield from filterfalse(is_normal, data)

**예제 5-5** 모두 함께 연결하기

In [ ]:
for itertools import islice

def filter_anomalous_data(data):
    data_group = groupby_day(data)
    yield from filter_anomalous_groups(data_group)

data = read_fake_data("fake_filename")
anomaly_generator = filter_anomalous_data(data)
first_five_anomalies = islice(anomaly_generator, 5)

for data_anomaly in first_five_anomalies:
    start_date = data_anomaly[0][0]
    end_date = data_anomaly[-1][0]
    print(f"Anomaly from {start_date} - {end_date}")

(뭔 말인지는 알겠는데... 내가 이걸 쓸 수 있을까...? help!!)